In [2]:
!pip install spotipy pandas

  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl (11.6 MB)
Using cached numpy-2.2.3-cp310-cp310-win_amd64.whl (12.9 MB)
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import api_config as conf


# Authenticate with Spotify
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=conf.SPOTIFY_CLIENT_ID,  # Replace with your Spotify client ID
    client_secret=conf.SPOTIFY_CLIENT_SECRET  # Replace with your Spotify client secret
))

# Search for a track
track_name = "Blinding Lights"
results = sp.search(q=track_name, type="track", limit=5)

# Print track details
for idx, track in enumerate(results["tracks"]["items"]):
    print(f"{idx + 1}: {track['name']} by {', '.join(artist['name'] for artist in track['artists'])}")
    print(f"   Album: {track['album']['name']}")
    print(f"   Preview URL: {track['preview_url']}")
    print(f"   Spotify URL: {track['external_urls']['spotify']}")


1: Blinding Lights by The Weeknd
   Album: After Hours
   Preview URL: None
   Spotify URL: https://open.spotify.com/track/0VjIjW4GlUZAMYd2vXMi3b
2: Blinding Lights by The Northern Lights Quintet
   Album: Songs Vol. 2
   Preview URL: None
   Spotify URL: https://open.spotify.com/track/1IyG1KwqwDisliFkCnpdvr
3: Blinding Lights by The Weeknd
   Album: Blinding Lights
   Preview URL: None
   Spotify URL: https://open.spotify.com/track/6qYkmqFsXbj8CQjAdbYz07
4: Blinding Lights by The Weeknd
   Album: The Highlights (Deluxe)
   Preview URL: None
   Spotify URL: https://open.spotify.com/track/04948IGlqY1vSh7AHbueiQ
5: Levitating by Dua Lipa
   Album: Future Nostalgia
   Preview URL: None
   Spotify URL: https://open.spotify.com/track/39LLxExYz6ewLAcYrzQQyP


In [5]:
top_20_spotify_genres = [
    "pop", "hip-hop", "rock", "r-n-b", "edm", "indie", "latin", "k-pop",
    "jazz", "reggae", "metal", "alternative", "dance", "house", "classical",
    "singer-songwriter", "punk", "soul", "trance", "techno"
]

len(top_20_spotify_genres)

20

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import api_config as conf
import pandas as pd
import time

# Authenticate with Spotify
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=conf.SPOTIFY_CLIENT_ID,  # Replace with your Spotify client ID
    client_secret=conf.SPOTIFY_CLIENT_SECRET  # Replace with your Spotify client secret
))

# Function to search for tracks by genre with pagination
def search_tracks_by_genre(genre, total_limit=1000, batch_size=50):
    tracks = []
    for offset in range(0, total_limit, batch_size):
        print(f"Fetching tracks for genre: {genre}, Offset: {offset}")
        results = sp.search(q=f"genre:{genre}", type="track", limit=batch_size, offset=offset)
        if not results["tracks"]["items"]:  # Stop if no more tracks are available
            break
        for track in results["tracks"]["items"]:
            tracks.append({
                "id": track['id'],
                "track_name": track["name"],
                "artist": ", ".join(artist["name"] for artist in track["artists"]),
                "album": track["album"]["name"],
                "popularity": track["popularity"],
                "release_date": track["album"]["release_date"],
                "spotify_url": track["external_urls"]["spotify"],
                "preview_url": track["preview_url"]
            })
        time.sleep(1)  # Sleep to avoid rate limiting
    return tracks

# Define genres and retrieve data
genres = top_20_spotify_genres
# genres = ["Jazz", "Pop", "Rock", "Hip-Hop/Rap", "Electronic/Dance"]  # List of genres to search
all_tracks = []

for genre in genres:
    try:
        genre_tracks = search_tracks_by_genre(genre, total_limit=500)
        for track in genre_tracks:
            track["Genre"] = genre  # Add genre column
        all_tracks.extend(genre_tracks)
    except:
        print("query error!")

# Convert to DataFrame
df = pd.DataFrame(all_tracks)

# Display results
print(f"Total tracks fetched: {len(df)}")
df

In [8]:
df.to_csv('raw_spotify.csv', index=False)

In [7]:
df['id'].describe()

count                      10000
unique                      7936
top       4dyx5SzxPPaD8xQIid5Wjj
freq                           6
Name: id, dtype: object

In [14]:
df.sample(5)

,id,track_name,artist,album,popularity,release Date,spotify_url,preview_url,Genre
624,37cRCP2V3367tAbWA5PMok,Dime in the Payphone,Tate Jackson Trio,Dime in the Payphone,52,2024-08-30,https://open.spotify.com/track/37cRCP2V3367tAb...,None,jazz
1580,3rdAz1fbUfZxYgaCviYhRo,Todo De Ti,Rauw Alejandro,VICE VERSA,79,2021-12-10,https://open.spotify.com/track/3rdAz1fbUfZxYga...,None,pop
1234,00RLNHc2jkEjUoCUlFgPVT,welcome and goodbye,"Dream, Ivory",welcome and goodbye,81,2018-03-31,https://open.spotify.com/track/00RLNHc2jkEjUoC...,None,pop
2120,07q0QVgO56EorrSGHC48y3,I Was Made For Lovin' You,KISS,Dynasty,81,1979-03-23,https://open.spotify.com/track/07q0QVgO56EorrS...,None,rock
3711,1bxLXzzgL7uBMbhtNCgwfu,#2,Aphex Twin,Selected Ambient Works Volume II,35,1994-03-07,https://open.spotify.com/track/1bxLXzzgL7uBMbh...,None,electronic/dance


In [16]:
import pandas as pd
df = pd.read_csv('./datasets/raw_spotify.csv')

In [18]:
for i in df['release_date']:
    print(i)

2024-11-11
2024-10-03
2024-07-19
2024-08-15
2024-07-23
2024-07-23
2024-08-16
2024-05-17
2024-06-27
2024-07-23
2023-11-03
2023-10-02
2023-09-29
2023-11-03
2023-11-03
2023-06-23
2023-07-14
2023-11-03
2023-11-06
2016-11-17
2019-08-23
2022-12-09
2024-03-08
2024-05-30
2014-11-17
2022-07-15
2016-11-25
2024-04-05
2024-05-17
2024-04-12
2010-05-11
2024-07-26
2024-08-23
2022-01-21
2022-12-08
2024-07-19
2023-05-26
2017-05-05
2024-08-23
2016-11-24
2020-08-04
2017-03-03
2024-08-23
2024-03-25
2023-04-20
2023-10-27
2017-03-03
2022-09-16
2016-07-29
2014-06-21
2024-09-02
2024-08-23
2012-01-01
2024-01-26
2024-04-01
2017-07-14
2024-06-06
2000-07-10
2018-05-18
2010-05-11
2024-04-26
2024-05-27
2015-11-13
2024-08-09
2023-11-10
2024-08-19
2024-06-21
2022-12-19
2024-09-27
2024-07-26
2023-01-02
2021-03-19
2024-05-27
2024-04-18
2022-12-08
2023-09-15
2020-02-10
2013-01-01
2017-04-07
2015-08-28
2024-08-30
2023-04-20
2017-04-20
2023-07-07
2012-12-07
2014-08-22
2012-12-07
2020-03-20
2022-08-01
2008-06-02
2013-01-01

In [24]:
df[df['popularity']>90]

,id,track_name,artist,album,popularity,release_date,spotify_url,preview_url,Genre
6,2plbrEY59IikOBgBGLjaoe,Die With A Smile,"Lady Gaga, Bruno Mars",Die With A Smile,97,2024-08-16,https://open.spotify.com/track/2plbrEY59IikOBg...,NaN,pop
7,6dOtVTDdiauQNBQEDOtlAB,BIRDS OF A FEATHER,Billie Eilish,HIT ME HARD AND SOFT,94,2024-05-17,https://open.spotify.com/track/6dOtVTDdiauQNBQ...,NaN,pop
28,3QaPy1KgI7nu9FJEQUgn6h,WILDFLOWER,Billie Eilish,HIT ME HARD AND SOFT,91,2024-05-17,https://open.spotify.com/track/3QaPy1KgI7nu9FJ...,NaN,pop
503,6AI3ezQ4o3HUoP6Dhudph3,Not Like Us,Kendrick Lamar,Not Like Us,92,2024-05-04,https://open.spotify.com/track/6AI3ezQ4o3HUoP6...,NaN,hip-hop
6000,2plbrEY59IikOBgBGLjaoe,Die With A Smile,"Lady Gaga, Bruno Mars",Die With A Smile,97,2024-08-16,https://open.spotify.com/track/2plbrEY59IikOBg...,NaN,dance


In [26]:
new_df = df.drop_duplicates(subset=['track_name'])
new_df.shape

(7217, 9)

In [28]:
new_df.to_csv('./datasets/spotify.csv', index=False)